# Deep Learning Mini Project
# Implement Gender and Age Detection: Predict if a Person is Male or Female and Estimate Their Age

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
# Define constants
img_height = 128
img_width = 128
batch_size = 32
epochs = 6

In [5]:
df = pd.read_csv('UTKFace.csv')
df.head()

,Unnamed: 0,age,gender,ethnicity,image
0,2226,1,1,0,109 146 198 132 169 221 134 171 223 137 174 22...
1,2047,1,0,3,0 1 2 2 1 1 5 1 0 11 3 3 1 0 6 27 35 45 97 111...
2,2048,1,0,3,54 64 91 55 66 95 56 72 105 58 76 113 67 85 12...
3,2049,1,0,3,80 98 125 77 94 129 113 131 169 147 169 201 16...
4,2050,1,0,3,63 55 96 58 51 94 72 67 112 83 80 126 91 90 13...


In [11]:
# Load and clean dataset
df = pd.read_csv('UTKFace.csv')
df['age'] = df['age'].apply(lambda x: min(x, 100))

In [12]:
# Convert 'image' column to numpy arrays
def parse_image(img_str):
    pixels = np.array(img_str.split(), dtype='float32')
    return pixels

In [14]:
# Parse image column
X = np.stack(df['image'].apply(parse_image).values)

# Assume images are 48x48 grayscale
X = X.reshape(-1, 100, 75, 1) / 255.0

In [15]:
# Labels
y_gender = df['gender'].values
y_age = df['age'].values

# Combine outputs as one for simplicity
y = np.stack((y_gender, y_age), axis=1)

In [17]:
from sklearn.model_selection import train_test_split
# Split into train/val/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [25]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 75, 1)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),  # This will now flatten to 8960 (automatically!)
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(2)  # One neuron for gender, one for age
])


In [26]:
model.compile(
    optimizer='adam',
    loss=['binary_crossentropy', 'mse'],
    metrics=['accuracy', 'mae']
)


In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout, Dense

input_img = Input(shape=(100, 75, 1))

x = Conv2D(32, (3, 3), activation='relu')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)

# Two outputs
gender_output = Dense(1, activation='sigmoid', name='gender')(x)
age_output = Dense(1, activation='linear', name='age')(x)

model = Model(inputs=input_img, outputs=[gender_output, age_output])

In [30]:
model.compile(
    optimizer='adam',
    loss={'gender': 'binary_crossentropy', 'age': 'mse'},
    metrics={'gender': 'accuracy', 'age': 'mae'}
)

history = model.fit(
    X_train, {'gender': y_train[:, 0], 'age': y_train[:, 1]},
    validation_data=(X_val, {'gender': y_val[:, 0], 'age': y_val[:, 1]}),
    epochs=10,
    batch_size=32
)

Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 30s 44ms/step - age_loss: 455.0335 - age_mae: 16.2940 - gender_accuracy: 0.5234 - gender_loss: 0.7015 - loss: 455.7355 - val_age_loss: 267.3553 - val_age_mae: 12.8258 - val_gender_accuracy: 0.6651 - val_gender_loss: 0.6249 - val_loss: 269.5788
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 27s 46ms/step - age_loss: 248.5218 - age_mae: 12.1220 - gender_accuracy: 0.6535 - gender_loss: 0.6248 - loss: 249.1464 - val_age_loss: 226.9052 - val_age_mae: 11.6290 - val_gender_accuracy: 0.7579 - val_gender_loss: 0.5351 - val_loss: 228.1958
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - age_loss: 217.1036 - age_mae: 11.1778 - gender_accuracy: 0.7422 - gender_loss: 0.5382 - loss: 217.6418 - val_age_loss: 230.1740 - val_age_mae: 11.2059 - val_gender_accuracy: 0.7811 - val_gender_loss: 0.5031 - val_loss: 229.6346
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 27s 45ms/step - age_loss: 200.9292 - age_mae: 10.6042 - gender_accuracy: 0.7551 - gender_loss: 0.5089 - loss

In [31]:

# Evaluate model
loss = model.evaluate(X_test, [y_test[:, 0], y_test[:, 1]])
print(f"Test Gender Accuracy: {loss[3]:.2f}")
print(f"Test Age MAE: {loss[4]:.2f}")

75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - age_loss: 109.5981 - age_mae: 7.8451 - gender_accuracy: 0.7804 - gender_loss: 0.4445 - loss: 110.0710
Test Gender Accuracy: 8.07
Test Age MAE: 0.80
